## This notebook uses each one of the 52 prediction models trained on autoML.ipynb to generate a gaussian_kde. Applied on the validation split (half) for the supernovae from type IA.

In [ ]:
%load_ext autoreload
%autoreload 2
import os 
import sys
import h2o

import pandas as pd

In [ ]:
module_path = os.path.abspath(os.path.join('..','..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.modeling.functions_autoML import (read_models,
                                           gen_models_predictions,
                                           gen_gaussian_kde, 
                                           print_gaussian_kde,
                                           gen_df_gaussian_kde)

In [ ]:
df = pd.read_pickle("../../data/structured/df.pkl")
#IA Filter
df = df.loc[df.type_bool == True]

In [ ]:
BEST_MODEL = 'StackedEnsemble_AllModels_AutoML_20201223_115349.zip'

In [ ]:
h2o.init(ip="localhost", port=54323)

### Creating DataFrame with predictions

In [ ]:
df_predictions = gen_models_predictions(df, read_models('../../models/mojo_50_ensemble'))

In [ ]:
df_predictions

In [ ]:
best_model = h2o.import_mojo(f'../../models/mojo_50_ensemble/{BEST_MODEL}')
predictions = df_predictions.loc[:,df_predictions.columns[1]:df_predictions.columns[-1]].values.reshape(len(df_predictions),52)
y_best_pred = df_predictions[f'{BEST_MODEL}'].values
y_real = df_predictions['REDSHIFT_SPEC'].values

In [ ]:
for i in range(4):
    print_gaussian_kde(gen_gaussian_kde(predictions[i])[0], predictions[i], y_real[i], y_best_pred[i], df_predictions.index[i])

In [ ]:
df_kde = gen_df_gaussian_kde(df_predictions,
                             export_csv = True,
                             best_model = BEST_MODEL,
                             path = '../../data/enriched/pdf_redshift_preds.csv')

In [ ]:
df_kde

In [ ]:
len(df_kde['predictions'][0])